<a href="https://colab.research.google.com/github/SoumitriM/data-science/blob/master/movie_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error
import itertools
from sklearn.metrics import silhouette_samples, silhouette_score
from mpl_toolkits.axes_grid1 import make_axes_locatable
import itertools
%matplotlib inline


In [0]:
movies = pd.read_csv('https://raw.githubusercontent.com/gouravaich/k-means-clustering-movie-ratings/master/ml-latest-small/movies.csv')
movies.head(5)



In [0]:
ratings = pd.read_csv('https://raw.githubusercontent.com/gouravaich/k-means-clustering-movie-ratings/master/ml-latest-small/ratings.csv')
ratings.head(5)

In [0]:
print("No. of ratings is ",len(ratings),"for", len(movies),"movies")


In [0]:
def get_avg_genre_rating(ratings, movies, genres, column_names):
  avg_ratings = pd.DataFrame()
  for genre in genres:
    genre_movies = movies[movies['genres'].str.contains(genre)]
    print(genre_movies)
    return(genre_movies)
    #avg_genre_votes_per_user = ratings[ratings['movieId'].isin(genre_movies['movieId'])].loc[:, ['userId', 'rating']].groupby(['userId'])['rating'].mean().round(2)
genre_rating = get_avg_genre_rating(ratings, movies,['Comedy'],"hey")
  

In [0]:
# Function to get the genre ratings
def get_genre_ratings(ratings, movies, genres, column_names):
    genre_ratings = pd.DataFrame()
    for genre in genres:        
        genre_movies = movies[movies['genres'].str.contains(genre) ]
        avg_genre_votes_per_user = ratings[ratings['movieId'].isin(genre_movies['movieId'])].loc[:, ['userId', 'rating']].groupby(['userId'])['rating'].mean().round(2)
        
        genre_ratings = pd.concat([genre_ratings, avg_genre_votes_per_user], axis = 1)
        
    genre_ratings.columns = column_names
    return genre_ratings
# Calculate the average rating of romance and scifi movies
genre_ratings = get_genre_ratings(ratings, movies, ['Romance', 'Sci-Fi'], ['avg_romance_rating', 'avg_scifi_rating'])
genre_ratings.head()


In [0]:
# Function to get the biased dataset
def bias_genre_rating_dataset(genre_ratings, score_limit_1, score_limit_2):
    biased_dataset =    genre_ratings[((genre_ratings['avg_romance_rating'] < score_limit_1 - 0.2) & (genre_ratings['avg_scifi_rating'] > score_limit_2)) | ((genre_ratings['avg_scifi_rating'] < score_limit_1) & (genre_ratings['avg_romance_rating'] > score_limit_2))]
    biased_dataset = pd.concat([biased_dataset[:300], genre_ratings[:2]])
    biased_dataset = pd.DataFrame(biased_dataset.to_records())
    return biased_dataset
# Bias the dataset
biased_dataset = bias_genre_rating_dataset(genre_ratings, 3.2,2.5)
# Printing the resulting number of records & the head of the dataset
print( "Number of records: ", len(biased_dataset))
biased_dataset


In [0]:
# Defining the scatterplot drawing function
def draw_scatterplot(x_data, x_label, y_data, y_label):
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(111)
    plt.xlim(0, 5)
    plt.ylim(0, 5)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.scatter(x_data, y_data, s=30)
# Plot the scatterplot
draw_scatterplot(biased_dataset['avg_scifi_rating'],'Avg scifi rating', biased_dataset['avg_romance_rating'], 'Avg romance rating')


In [0]:
x = biased_dataset[['avg_scifi_rating','avg_romance_rating']].values
k_means = KMeans(n_clusters = 7)
predictions = k_means.fit_predict(x)
predictions
# Defining the cluster plotting function
def draw_clusters(biased_dataset, predictions):
    fig = plt.figure('Scatter')
    ax = fig.add_subplot(111)
    ax.set_xlabel('Avg scifi rating')
    ax.set_ylabel('Avg romance rating')
clustered = pd.concat([biased_dataset.reset_index(), pd.DataFrame({'group':predictions})], axis=1)
plt.scatter(clustered['avg_scifi_rating'], clustered['avg_romance_rating'], c=clustered['group'], s= 12)

In [0]:
from sklearn.metrics import silhouette_score
k_values = np.arange(2,len(x)+1,5)
def clustering_errors(k,data):
  predictions = KMeans(n_clusters = k).fit_predict(data)
  silhouette_avg = silhouette_score(data, predictions)
  return silhouette_avg
errors_per_k = [clustering_errors(k,x) for k in k_values]

  


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:972: ConvergenceWarning: Number of distinct clusters (178) found smaller than n_clusters (182). Possibly due to duplicate points in X.
  return_n_iter=True)
